Importing Models

In [ ]:
!pip install --upgrade transformers==4.38.0 accelerate datasets evaluate rouge-score sentencepiece


  Using cached transformers-4.38.0-py3-none-any.whl.metadata (131 kB)
  Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.38.0-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.0 which is incompatible.


In [ ]:
!pip install --upgrade peft
!pip install --upgrade transformers accelerate datasets evaluate rouge-score sentencepiece


  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.0
    Uninstalling transformers-4.38.0:
      Successfully uninstalled transformers-4.38.0


load the data

In [ ]:
import pandas as pd
from datasets import Dataset

import pandas as pd

df = pd.read_csv(
    "/content/Reviews.csv",
    on_bad_lines="skip",
    engine="python"
)

Remove short text and summaries

In [ ]:
df = df[['Text', 'Summary']].dropna().rename(columns={'Text': 'text', 'Summary': 'summary'})

df = df[df['text'].str.len() > 30]
df = df[df['summary'].str.len() > 5]


 Convert to Hugging Face Dataset and split

In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

Tokenization

In [ ]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["text"]]

    # Pad all inputs to max_length
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=64,
            truncation=True,
            padding="max_length"  # pad labels to fixed length
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/237230 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/26359 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
import evaluate

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 4) for k, v in result.items()}


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4, # Reduced from 8 to 4
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_total_limit=1,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    report_to=[]

)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#trainer.train() uncomment to fine tune

/tmp/ipython-input-3650975027.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.1
)

model = get_peft_model(model, config)

trainer.train()


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 776.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 38.12 MiB is free. Process 14094 has 14.70 GiB memory in use. Of the allocated memory 12.95 GiB is allocated by PyTorch, and 1.62 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Evaluation on Rouge

In [ ]:
import torch
from tqdm import tqdm
import evaluate

rouge = evaluate.load("rouge")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_texts = dataset["test"]["text"][:50]
reference_summaries = dataset["test"]["summary"][:50]

generated_summaries = []

print("Generating summaries...")
for text in tqdm(test_texts):
    inputs = tokenizer(
        "summarize: " + text,
        return_tensors="pt",
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=128,
            min_length=20,
            num_beams=2,
            do_sample=True,
            temperature=0.8,
            repetition_penalty=2.0,
            length_penalty=1.2,
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    generated_summaries.append(summary)

# Compute ROUGE
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)

print("\nROUGE Evaluation Results:")
for key in ["rouge1", "rouge2", "rougeL", "rougeLsum"]:
    score = results[key] * 100  # just multiply the float
    print(f"{key}: {score:.2f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import evaluate

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Compute ROUGE per example
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, ref in zip(generated_summaries, reference_summaries):
    result = rouge.compute(predictions=[pred], references=[ref])
    rouge1_scores.append(result["rouge1"])
    rouge2_scores.append(result["rouge2"])
    rougeL_scores.append(result["rougeL"])

# Plot histogram for ROUGE-1
plt.figure(figsize=(8,5))
sns.histplot(rouge1_scores, bins=10, kde=True, color="skyblue")
plt.title("Distribution of ROUGE-1 Scores")
plt.xlabel("ROUGE-1 Score")
plt.ylabel("Count")
plt.show()

# plot ROUGE-2 and ROUGE-L in one figure
plt.figure(figsize=(8,5))
sns.histplot(rouge2_scores, bins=10, kde=True, color="salmon", label="ROUGE-2")
sns.histplot(rougeL_scores, bins=10, kde=True, color="lightgreen", label="ROUGE-L")
plt.title("Distribution of ROUGE-2 and ROUGE-L Scores")
plt.xlabel("Score")
plt.ylabel("Count")
plt.legend()
plt.show()


Evaluation on BertScore

In [ ]:
!pip install bert_score
bertscore = evaluate.load("bertscore")
results = bertscore.compute(predictions=generated_summaries, references=reference_summaries, lang="en")
print(results["f1"])

In [ ]:
import evaluate
import matplotlib.pyplot as plt

# Load BERTScore metric
bertscore = evaluate.load("bertscore")

results = bertscore.compute(predictions=generated_summaries, references=reference_summaries, lang="en")

# Print F1 scores
print("BERTScore F1:", results["f1"])

# Visualize BERTScore
plt.figure(figsize=(8, 5))
plt.bar(range(len(results["f1"])), results["f1"], color='skyblue')
plt.xlabel("Summary Index")
plt.ylabel("BERTScore F1")
plt.title("BERTScore F1 for Generated Summaries")
plt.ylim(0, 1)
plt.show()


Inference

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_corpus = [
    """I ordered the grilled chicken sandwich, and it turned out better than I expected.
     The bread was toasted perfectly and had a nice crunch. The chicken was juicy and seasoned with
     herbs that added a great flavor. The fries on the side were crispy and not too oily.
     I would definitely recommend this meal to anyone who loves simple yet delicious food."""
]


inputs = tokenizer(["summarize: " + doc for doc in test_corpus], return_tensors="pt", max_length=512, truncation=True)

inputs = {k: v.to(device) for k, v in inputs.items()}

# generate summary
with torch.no_grad():
   summary_ids = model.generate(
    **inputs,
    max_length=80,
    min_length=20,
    num_beams=4,
    do_sample=True,
    temperature=0.7,
    repetition_penalty=2.0,
    length_penalty=1.0,
    early_stopping=True
)

print("Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))

Save the Model

In [ ]:
trainer.save_model("./t5-summarizer_results")
tokenizer.save_pretrained("./t5-summarizer_results")